In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
pd.set_option('max_colwidth', 275)

In [2]:
#importing the Train Data using pandas
df = pd.read_table('Data/Train_Data.txt')
df

,sentiment,review
0,1,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set i..."
1,1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not o..."
2,1,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected seria..."
3,-1,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first o..."
4,1,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter...."
...,...,...
14994,1,"*** out of ****<br /><br />Yep! Dressed To Kill is that kind of a movie. It's like Kalifornia, but it's different. Remember? That movie from 1993 which stars Brad Pitt as a serial killer who is ""welcomed"" by a couple of travelers in a trip to California as a buddy who mi..."
14995,-1,"Bobcat Goldthwait should be commended for attempting to do something different with this surprisingly heartfelt film, a cautionary tale about the pitfalls of being honest about everything. Melinda Hamilton stars as Amy, a girl who has had oral sex with a canine in the pa..."
14996,1,"And it's not because since her days on ""Clarissa Explains It All"" that I've had a bit of a crush on Melissa Joan Hart, who at the time this show was popular was already well into her 20s, but was still able to get teenage roles. ""Sabrina, the Teenage Witch"" was Hart's ne..."
14997,-1,"A traveling couple (Horton and Hamilton)stumble onto the town of Gatlin, where kids have slaughtered the adults and are always eager to slaughter more, along the way they're separated and Peter Horton of course must save his wife from these tiresome, er terrible tykes in..."


In [3]:
# I want to have all the reviews from the data frame
all_reviews = df['review']
all_reviews.head()

0    One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set i...
1    A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not o...
2    I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected seria...
3    Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie i

In [4]:
# Here I will be cleaning the data from HTML tags, puntuation, stop words, and stem words.
# I will also be splititng words into sentences 
clean = re.compile('<.*?>')
clean_reviews = df['review'].str.replace(clean,'',regex=True)
clean_reviews

0        One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from...
1        A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only "has got all the pol...
2        I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected seria...
3        Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.This mov

In [5]:
from nltk import sent_tokenize
#sentences = clean_reviews.apply(sent_tokenize(clean_reviews))